In [ ]:
#upload package
import pandas as pd
import numpy as np
import time
import scipy
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Seed for reproducibility
np.random.seed(0)

In [2]:
#upload dataset
train_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/SVM/bank-note/train.csv", header = None, names = ['variance','skewness','curtosis','entropy','y'])
test_data = pd.read_csv("D:/EIC-Code/00-Python/Machine-Learning-HW/SVM/bank-note/test.csv", names = ['variance','skewness','curtosis','entropy','y'])

features = ['variance','skewness','curtosis','entropy']
outcome = 'y'

X_train = train_data[features].values #change to matrix multiple
y_train = train_data[outcome].values
X_test = test_data[features].values
y_test = test_data[outcome].values
y_train[y_train == 0] = -1
y_test[y_test == 0] = -1

In [ ]:
class PrimalSVM:
    def __init__(self, gamma, a, C, N):
        self.gamma = gamma
        self.a = a
        self.C = C
        self.N = N
        self.w = None  # Weight vector
        self.b = 0  # Bias term
        self.objective_curve = []  # Stores hinge loss at each epoch

    def _hinge_loss(self, X, y):
        loss = 0.5 * np.dot(self.w, self.w) + self.C * np.sum(np.maximum(0, 1 - y * (np.dot(X, self.w) + self.b))) #objective function
        return loss

    def fit(self, X_train, y_train, epochs, schedule):
        n_samples, n_features = X_train.shape
        self.w = np.zeros(n_features)  # Initialize weights

        for epoch in range(epochs):
            # Shuffle training data
            perm = np.random.permutation(n_samples)
            X_train, y_train = X_train[perm], y_train[perm]

            for i, (xi, yi) in enumerate(zip(X_train, y_train)):
                t = epoch * n_samples + i + 1  # Global step count

                # Learning rate schedule
                if schedule == "schedule1":
                    eta_t = self.gamma / (1 + (self.gamma / self.a) * t)
                elif schedule == "schedule2":
                    eta_t = self.gamma / (1 + t)
                else:
                    raise ValueError("Invalid schedule. Choose 'schedule1' or 'schedule2'.")

                # Sub-gradient updates
                if yi * (np.dot(self.w, xi) + self.b) <= 1:
                    self.w = (1 - eta_t) * self.w + eta_t * self.C * self.N * yi * xi
                    self.b += eta_t * self.C * yi
                else:
                    self.w = (1 - eta_t) * self.w

            # Compute hinge loss at the end of each epoch
            self.objective_curve.append(self._hinge_loss(X_train, y_train))
        

    def predict(self, X):
        return np.sign(np.dot(X, self.w) + self.b)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions != y)

    def get_objective_curve(self):
        return self.objective_curve
    
    def get_weights(self):
        return self.w
    
    def get_bias(self):
        return self.b


In [ ]:
# Define hyperparameter search space
Cs = [100 / 873, 500 / 873, 700 / 873]
gamma_values = [1, 0.5, 0.1, 0.01, 0.001]
a_values = [1, 10, 50, 100]
N = 1
epochs = 100

# Initialize variables to track the best parameters and lowest error for each schedule and C
best_params_per_schedule = {"schedule1": {}, "schedule2": {}}

# Perform grid search
for schedule in ["schedule1", "schedule2"]:
    for C in Cs:
        lowest_error_for_C = float("inf")
        best_params_for_C = None
        for gamma in gamma_values:
            for a in a_values:
                # Initialize and train the SVM
                svm = PrimalSVM(gamma=gamma, a=a, C=C, N=N)
                svm.fit(X_train, y_train, epochs=epochs, schedule=schedule)

                # Calculate training and test errors
                train_error = svm.score(X_train, y_train)
                test_error = svm.score(X_test, y_test)
                weights = svm.get_weights()
                bias = svm.get_bias()

                #print(f"Schedule: {schedule}, C={C:.6f}, gamma={gamma}, a={a}, "
                #f"train_error: {train_error:.4f}, test_error: {test_error:.4f},"
                #f"weights, {weights}, bias = {bias}")

                #output objective curve 
                #objective_curve = svm.get_objective_curve()
                #plt.figure(figsize=(8, 5))
                #plt.plot(range(1, len(objective_curve) + 1), objective_curve, marker='o')
                #plt.title("Objective Function Curve (Hinge Loss)")
                #plt.xlabel("Epoch")
                #plt.ylabel("Hinge Loss")
                #plt.grid(True)
                #plt.show()

                # Update the best parameters for the current C
                if test_error < lowest_error_for_C:
                    lowest_error_for_C = test_error
                    best_params_for_C = {
                        "C": C,
                        "gamma": gamma,
                        "a": a,
                        "schedule": schedule,
                        "train_error": train_error,
                        "test_error": test_error,
                        "weights": weights,
                        "bias": bias
                    }
        
        # Store the best parameters for the current C and schedule
        best_params_per_schedule[schedule][C] = best_params_for_C

# Print the best parameters for each C, separated by schedule
for schedule, params_per_C in best_params_per_schedule.items():
    print(f"\nBest Parameters for {schedule}:")
    for C, params in params_per_C.items():
        print(f"C={C:.6f}, Schedule: {params['schedule']}, gamma={params['gamma']}, a={params['a']}, "
              f"train_error: {params['train_error']:.4f}, test_error: {params['test_error']:.4f}, "
              f"weights: {params['weights']}, bias: {params['bias']}")



Best Parameters for schedule1:
C=0.114548, Schedule: schedule1, gamma=0.5, a=100, train_error: 0.0608, test_error: 0.0840, weights: [-0.15217176 -0.11147137 -0.05579284 -0.01562807], bias: 0.14298246924389865
C=0.572738, Schedule: schedule1, gamma=0.01, a=1, train_error: 0.0241, test_error: 0.0180, weights: [-0.32441602 -0.17212906 -0.18608215 -0.02517079], bias: 0.5004602638064877
C=0.801833, Schedule: schedule1, gamma=0.001, a=100, train_error: 0.0183, test_error: 0.0160, weights: [-0.34762413 -0.18979735 -0.22631406 -0.00247417], bias: 0.6468729605176408

Best Parameters for schedule2:
C=0.114548, Schedule: schedule2, gamma=1, a=50, train_error: 0.0631, test_error: 0.0860, weights: [-0.15368874 -0.11439085 -0.05609331 -0.01529727], bias: 0.1352745796472408
C=0.572738, Schedule: schedule2, gamma=1, a=10, train_error: 0.0218, test_error: 0.0160, weights: [-0.32397074 -0.17102877 -0.18428299 -0.02530887], bias: 0.4902974613772339
C=0.801833, Schedule: schedule2, gamma=1, a=10, train_e

In [ ]:
class DualSVM:
    def __init__(self, C):
        self.C = C  # Regularization parameter
        self.alpha = None  # Lagrange multipliers
        self.w = None  # Weight vector
        self.b = None  # Bias term


    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        # kernel matrix (linear kernel)
        K = np.dot(X, X.T)

        # define the dual objective function
        def objective(alpha):
            return -np.sum(alpha) + 0.5 * np.sum((alpha * y)[:, None] * (alpha * y) * K)

        # Initial guess for alpha
        alpha0 = np.zeros(n_samples)

        # Bounds for alpha: 0 <= alpha <= C
        bounds = [(0, C) for _ in range(n_samples)]

        # Equality constraint: sum(alpha * y) = 0
        constraints = {
            'type': 'eq',
            'fun': lambda alpha: np.dot(alpha, y),
            'jac': lambda alpha: y
            }
        
        # Solve the optimization problem
        result = minimize(
            objective,
            alpha0,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )

        # extract the optimal alpha
        self.alpha = result.x #minimize the function to get alpha

        # compute weight vector
        self.w = np.sum((self.alpha * y)[:, None] * X, axis=0)

        # compute bias term
        support_vector_idx = np.where((self.alpha > 0) & (self.alpha < self.C))[0][0]
        self.b = y[support_vector_idx] - np.dot(self.w, X[support_vector_idx])

    def predict(self, X):
        return np.sign(np.dot(X, self.w) + self.b)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions != y)

In [ ]:

# Define hyperparameter C
Cs = [100 / 873, 500 / 873, 700 / 873] #, 873

# Train and evaluate the model for different values of C
for C in Cs:
    svm = DualSVM(C)
    svm.fit(X_train, y_train)
    train_error = svm.score(X_train, y_train)
    test_error = svm.score(X_test, y_test)
    print(f"C={C}, Train error={train_error:.2f}, Test error={test_error:.2f}")
    print(f"Weights:, {svm.w}, Bias:, {svm.b}")

# Print the weights and bias for the best model
print("Weights:", svm.w)
print("Bias:", svm.b)

d:\EID-Download\02 Academic\Python38\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


C=0.1145475372279496, Train Accuracy=0.11, Test Accuracy=0.12
Weights:, [-0.94292598 -0.65149184 -0.73372197 -0.04102195], Bias:, 4.14119177534919
C=0.572737686139748, Train Accuracy=0.15, Test Accuracy=0.14
Weights:, [-1.56393784 -1.01405165 -1.18065044 -0.15651687], Bias:, 7.590350666124916
C=0.8018327605956472, Train Accuracy=0.40, Test Accuracy=0.43
Weights:, [-2.04254833 -1.28068891 -1.51351532 -0.24905307], Bias:, 12.975949611428163
Weights: [-2.04254833 -1.28068891 -1.51351532 -0.24905307]
Bias: 12.975949611428163


In [ ]:
class GaussianSVM:
    def __init__(self, C, gamma):
        self.C = C
        self.gamma = gamma
        self.alpha = None  # lagrange multipliers
        self.b = None  # bias term
        self.X_train = None  # training features
        self.y_train = None  # training labels

    def gaussian_kernel(self, x1, x2):
        return np.exp(-np.linalg.norm(x1 - x2) ** 2 / self.gamma)

    def kernel_matrix(self, X):
        n_samples = X.shape[0]
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i, j] = self.gaussian_kernel(X[i], X[j])
        return K

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        n_samples, n_features = X.shape

        # Compute the kernel matrix
        K = self.kernel_matrix(X)

        # Define the dual objective function
        def objective(alpha):
            return -np.sum(alpha) + 0.5 * np.sum((alpha * y)[:, None] * (alpha * y) * K)

        # Bounds for alpha: 0 <= alpha <= C
        bounds = [(0, self.C) for _ in range(n_samples)]

        # Equality constraint: sum(alpha * y) = 0
        constraints = {
            'type': 'eq',
            'fun': lambda alpha: np.dot(alpha, y),
            'jac': lambda alpha: y
        }

        # Initial guess for alpha
        alpha0 = np.zeros(n_samples)

        # Solve the optimization problem
        result = minimize(
            objective,
            alpha0,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints,
            options={'maxiter': 1000, 'disp': True}
        )

        # Extract the optimal alpha
        self.alpha = result.x

        # Compute bias term using support vectors
        support_vector_idx = np.where((self.alpha > 1e-4) & (self.alpha < self.C))[0]
        support_vector_idx = support_vector_idx[0]
        self.b = y[support_vector_idx] - np.sum(self.alpha * y * K[support_vector_idx])

    def predict(self, X):
        y_pred = []
        for x in X:
            # Decision function
            decision = np.sum(
                self.alpha * self.y_train *
                np.array([self.gaussian_kernel(x, x_train) for x_train in self.X_train])
            ) + self.b
            y_pred.append(np.sign(decision))
        return np.array(y_pred)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions != y)  # Error rate

    def get_support_vectors(self):
        return np.where((self.alpha > 1e-4) & (self.alpha < self.C))[0]

In [ ]:
Cs = [100 / 873, 500 / 873, 700 / 873] #, 873
gammas = [0.1, 0.5, 1, 5, 100]

lowest_error = float("inf")
best_params = None

for C in Cs:
    for gamma in gammas:
        gsvm = GaussianSVM(C, gamma)
        gsvm.fit(X_train, y_train)
        sv_indices = gsvm.get_support_vectors()
        train_error = gsvm.score(X_train, y_train)
        test_error = gsvm.score(X_test, y_test)
        print(f"C={C:.4f}, gamma={gamma}, Train error={train_error:.4f}, Test error={test_error:.4f}, Number of Support Vectors: {len(sv_indices)}")

        if test_error < lowest_error:
            lowest_error = test_error
            best_params = (C, gamma)

print(f"Best Parameters: C={best_params[0]:.4f}, gamma={best_params[1]}")
print(f"Best Test Error: {lowest_error:.4f}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: -82.72279493439663
            Iterations: 15
            Function evaluations: 13098
            Gradient evaluations: 15
C=0.1145, gamma=0.1, Train error=0.4461, Test error=0.4420, Number of Support Vectors: 853
Optimization terminated successfully    (Exit mode 0)
            Current function value: -74.16551934430456
            Iterations: 23
            Function evaluations: 20084
            Gradient evaluations: 23
C=0.1145, gamma=0.5, Train error=0.4071, Test error=0.4260, Number of Support Vectors: 745
Optimization terminated successfully    (Exit mode 0)
            Current function value: -63.41453356303385
            Iterations: 25
            Function evaluations: 21830
            Gradient evaluations: 25
C=0.1145, gamma=1, Train error=0.2144, Test error=0.2720, Number of Support Vectors: 655


d:\EID-Download\02 Academic\Python38\lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -26.45826553383153
            Iterations: 36
            Function evaluations: 31437
            Gradient evaluations: 36
C=0.1145, gamma=5, Train error=0.0034, Test error=0.0040, Number of Support Vectors: 405
Optimization terminated successfully    (Exit mode 0)
            Current function value: -20.289226978400574
            Iterations: 26
            Function evaluations: 22703
            Gradient evaluations: 26
C=0.1145, gamma=100, Train error=0.0138, Test error=0.0080, Number of Support Vectors: 38
Optimization terminated successfully    (Exit mode 0)
            Current function value: -285.8328142581335
            Iterations: 19
            Function evaluations: 16590
            Gradient evaluations: 19
C=0.5727, gamma=0.1, Train error=0.0000, Test error=0.3480, Number of Support Vectors: 794
Optimization terminated successfully    (Exit mode 0)
            Current function value: 

In [ ]:
# Hyperparameters
C = 500 / 873 
gammas = [0.01, 0.1, 0.5, 1, 5]

# Track support vectors
support_vectors = {}
for gamma in gammas:
    gsvm = GaussianSVM(C=C, gamma=gamma)
    gsvm.fit(X_train, y_train)
    sv_indices = gsvm.get_support_vectors()
    support_vectors[gamma] = sv_indices
    print(f"Gamma={gamma}, Number of Support Vectors: {len(sv_indices)}")

# Calculate overlaps between consecutive gammas
for i in range(len(gammas) - 1):
    gamma1, gamma2 = gammas[i], gammas[i + 1]
    overlap = len(np.intersect1d(support_vectors[gamma1], support_vectors[gamma2]))
    print(f"Overlap between gamma={gamma1} and gamma={gamma2}: {overlap}")

Optimization terminated successfully    (Exit mode 0)
            Current function value: -326.98871956857363
            Iterations: 9
            Function evaluations: 7858
            Gradient evaluations: 9
Gamma=0.01, Number of Support Vectors: 487
Optimization terminated successfully    (Exit mode 0)
            Current function value: -285.8328142581335
            Iterations: 19
            Function evaluations: 16590
            Gradient evaluations: 19
Gamma=0.1, Number of Support Vectors: 794
Optimization terminated successfully    (Exit mode 0)
            Current function value: -159.75505159853384
            Iterations: 39
            Function evaluations: 34053
            Gradient evaluations: 39
Gamma=0.5, Number of Support Vectors: 613
Optimization terminated successfully    (Exit mode 0)
            Current function value: -102.93934454458342
            Iterations: 53
            Function evaluations: 46277
            Gradient evaluations: 53
Gamma=1, Number of Su

In [26]:
class KernelPerceptron:
    def __init__(self, gamma, max_epochs=10):
        self.gamma = gamma
        self.max_epochs = max_epochs
        self.c = None
        self.X_train = None
        self.y_train = None

    def gaussian_kernel(self, x1, x2):
        return np.exp(-np.linalg.norm(x1 - x2)**2 / self.gamma)

    def fit(self, X, y):
        N = X.shape[0]
        self.c = np.zeros(N)  # Mistake counts
        self.X_train = X
        self.y_train = y

        for epoch in range(self.max_epochs):
            for i in range(N):
                # Compute decision function
                decision = sum(self.c[j] * y[j] * self.gaussian_kernel(X[j], X[i]) for j in range(N))
                if y[i] * decision <= 0:  # Misclassified
                    self.c[i] += 1  # Update mistake count

    def predict(self, X):
        y_pred = []
        for x in X:
            # Compute decision function for new point
            decision = sum(self.c[i] * self.y_train[i] * self.gaussian_kernel(self.X_train[i], x) for i in range(len(self.X_train)))
            y_pred.append(np.sign(decision))
        return np.array(y_pred)

    def score(self, X, y):
        predictions = self.predict(X)
        return np.mean(predictions != y)  # Accuracy


In [ ]:
# Test different gamma values
gammas = [0.1, 0.5, 1, 5, 100]
for gamma in gammas:
    kp = KernelPerceptron(gamma=gamma)
    kp.fit(X_train, y_train)
    train_error = kp.score(X_train, y_train)
    test_error = kp.score(X_test, y_test)
    print(f"Gamma={gamma}, Train error ={train_error:.4f}, Test error ={test_error:.4f}")


Gamma=0.1, Train error =0.0000, Test error =0.0020
Gamma=0.5, Train error =0.0000, Test error =0.0040
Gamma=1, Train error =0.0000, Test error =0.0040
Gamma=5, Train error =0.0000, Test error =0.0040
Gamma=100, Train error =0.0000, Test error =0.0000
